In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from api import get_neo_data

: 

API-Schlüssel

In [ ]:
api_key = "0m4vbWgrfhHHwXv7FS1U51c3TNxlFqZNcNCUuWs2"
start_date = "2024-03-16"  # Beispielstartdatum
end_date = "2024-03-23"  # Beispielenddatum
neo_data = get_neo_data(api_key, start_date, end_date)

Extrahiere relevante Merkmale aus den API-Daten

In [ ]:
features = []
labels = []

for data in neo_data:
    near_earth_objects = data.get('near_earth_objects', {})
    for date, asteroids in near_earth_objects.items():
        for asteroid in asteroids:
            features.append([
                asteroid['absolute_magnitude_h'],
                asteroid['estimated_diameter']['kilometers']['estimated_diameter_min'],
                asteroid['estimated_diameter']['kilometers']['estimated_diameter_max'],
                asteroid['close_approach_data'][0]['miss_distance']['kilometers'],
                asteroid['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']
            ])
            labels.append(asteroid['is_potentially_hazardous_asteroid'])

Erstelle einen DataFrame aus den extrahierten Merkmalen und Labels

In [ ]:
df = pd.DataFrame(features, columns=['absolute_magnitude_h', 'min_diameter_km', 'max_diameter_km', 'miss_distance_km', 'relative_velocity_km_hour'])
df['is_potentially_hazardous'] = labels

Aufteilung der Daten in Trainings- und Testsets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('is_potentially_hazardous', axis=1), df['is_potentially_hazardous'], test_size=0.2, random_state=42)

Modelltraining

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

Vorhersage auf dem Testset

In [ ]:
predictions = model.predict(X_test)

Evaluierung des Modells

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)